In [3]:
import ROOT
from ROOT import *
from array import array
import math

# Setting what to be shown in statistics box:
gStyle.SetOptStat(1110)   # Print for stat: Entries, Mean, and RMS
gStyle.SetOptFit(1111)    # Print for fit:  Everything!
#test
gStyle.SetStatX(0.82);    # Top left corner.
gStyle.SetStatY(0.86);

In [7]:
#----------------------------------------------------------------------
# Read data
#----------------------------------------------------------------------
#name    z    F    sigF    tau_cent,rest,B    dtau+    dtau-

name=[]
z             = array("f", [])
F             = array("f", [])
dF            = array("f", [])
tau_centrestB = array("f", [])
dTaup         = array("f", [])
dTaum         = array("f", [])

count=0
with open( 'Data.txt', 'r' ) as infile :
    for line in infile:
        line = line.strip().split()
        navn.append((line[0]))
        z.append(float(line[1]))
        F.append(float(line[2]))
        dF.append(float(line[3]))
        tau_centrestB.append(float(line[4]))
        dTaup.append(float(line[5]))
        dTaum.append(float(line[6]))

        #RBLR.append(float(line[8]))
        # Print the numbers as a sanity check
        if count<5: print "  Read data:  navn: %s, z: %6.3f, F:  %6.3f +- %6.3f, tau: %6.3f + %6.3f -%6.3f"%((line[0]), float(line[1]), float(line[2]), float(line[3]),float(line[4]),float(line[5]),float(line[6]))
        count+=1
Nobs = len(name)
#print "Nobs: ", Nobs
c    = 299792458. #m/s
H0   = 2.23 *10.**(-18) # s^-1       svarer til +- 69 (km/s)/Mpc
dH0  = 0.03 *10.**(-18) # s^-1       svarer til +- 1 (km/s)/Mpc
iWithTau  = [i for i,x in enumerate(tau_centrestB) if x>-5] #indices with a value for tau. The code won't work if it's negative


  Read data:  navn: Mrk142, z:  0.045, F:   0.928 +-  0.080, tau:  2.740 +  0.730 - 0.830
  Read data:  navn: SBS1116+583A, z:  0.028, F:   0.262 +-  0.028, tau:  2.310 +  0.620 - 0.490
  Read data:  navn: Arp151, z:  0.021, F:   0.059 +-  0.860, tau:  0.150 +  3.990 - 0.490
  Read data:  navn: Mrk1310, z:  0.019, F:   0.495 +-  0.005, tau:  3.660 +  0.590 - 0.610
  Read data:  navn: Mrk202, z:  0.021, F:   0.299 +-  0.027, tau:  3.050 +  1.730 - 1.120


In [9]:
#----------------------------------------------------------------------
# Analyse data
#----------------------------------------------------------------------
#tauF vs. z
#F er givet i erg s^-1 cm^-2 = 10^-3 kg/s^3
F    = array("f", [x*10**3 for x in F]) # Nu i kg/s^3

# tauF vs. z
tauF      = array('f', [tau_centrestB[i]/math.sqrt(F[i]) for i in iWithTau])   #F/sqrt(tau)

#Slightly wrong uncertainties:
dtaumean  = array('f', [(dTaum[i]+dTaup[i])/2. for i,x in enumerate(tau_centrestB)]) #Mean of errors of each side
dtauF     = array('f', [math.sqrt(1./F[i] * dtaumean[i]**2 + (tau_centrestB[i] * F[i]**(-3./2.) /2. )**2 * dF[i]**2) for i in iWithTau] )
dz        = array('f', [0.001]*len(z)) #That order of magnitude

#print "tauF:", tauF, "+-",dtauF
#print "z:", z, "+-",dz

#Values with a defined tau
zWithTau  = array('f',[z[x] for x in iWithTau])
dzWithTau = array('f',[dz[x] for x in iWithTau])

#print "zWithTau:", zWithTau, "+-",dzWithTau

#-------------------------------------------------------------------------------------
# L vs. R
RBLR      = array("f", [x*c for x in tau_centrestB]) #R=c*tau
v         = array("f", [-((x-1.)**2-1.)/((x+1.)**2+1.)*c for x in z]) # Cosmological redshift -> v
D         = array('f', [x/H0 for x in v]) #Distance til objekt i m. v=H0*D
L         = array('f', [F[i]*4*math.pi*(D[i]**2)**2 for i,x in enumerate(F)]) #L=F*4piD^2 i watt

dRBLR     = array("f", [x*c for x in dtaumean])  # dR=c*dtau
dv        = array("f", [dz[i]*2*c * (1+x)/(1.+(1.+x)**2) * (1 - (1+x)**2.-1)/((1.+x)**2+1) for i,x in enumerate(z)]) #Error propagation, tjek evt. om forsimplet giver ca. det samme
dD        = array("f", [math.sqrt(1./H0**2.*dv[i]**2.+(v[i]/H0)**2*dH0**2) for i,x in enumerate(v)])
dL        = array("f", [4*math.pi*math.sqrt(D[i]**4*dF[i]**2+(2*F[i]*D[i])**2*dD[i]**2) for i,x in enumerate(D)])

#Values with a defined tau
LWithTau  = array("f", [L[x] for x in iWithTau])
dLWithTau = array("f", [dL[x] for x in iWithTau])

print "RBLR:", RBLR, "+-",dRBLR
print "v:", v
print "D:", D
print "LWithTau:", LWithTau, "+-",dLWithTau

RBLR: array('f', [821431360.0, 692520576.0, 44968872.0, 1097240448.0, 914366976.0, 1846721536.0, 1663848192.0, -2997924608.0, -2997924608.0, 1253132416.0, -2997924608.0, -2997924608.0, 1990621824.0]) +- array('f', [233838112.0, 166384816.0, 671535104.0, 179875488.0, 427204224.0, 422707392.0, 575601536.0, -2997924608.0, -2997924608.0, 323775840.0, -2997924608.0, -2997924608.0, 265316320.0])
v: array('f', [12591371.0, 8012369.0, 6125404.0, 5651733.5, 6105716.5, 3801781.25, 4290649.0, 5629117.5, 2296521.25, 5019221.0, 8482094.0, 9592852.0, 1549754.625])
D: array('f', [5.64635467014972e+24, 3.5929905176906316e+24, 2.7468179026728677e+24, 2.5344096975020014e+24, 2.737989406251341e+24, 1.7048346845797457e+24, 1.924057808764343e+24, 2.524268023486727e+24, 1.0298301283978731e+24, 2.2507716959335148e+24, 3.8036295648126787e+24, 4.3017273953694794e+24, 6.949572319133808e+23])
LWithTau: array('f', [inf, inf, inf, inf, inf, inf, inf, inf, inf]) +- array('f', [inf, inf, inf, inf, inf, inf, inf, inf

In [10]:
#----------------------------------------------------------------------
# Plot data
#----------------------------------------------------------------------
# tauFz
Graph_tauFz = TGraphErrors(len(iWithTau), zWithTau, tauF, dzWithTau, dtauF)

canvas = TCanvas("canvas","", 50, 50, 600, 400)


fit = TF1("fit", "[0]+[1]*x" , 0, 0.05) #
fit.SetParameters(4., 0.1) #Start
fit.SetNpx(500) #good resolution
Graph_tauFz.Fit("fit", "R")

Graph_tauFz.GetXaxis().SetTitle("z")
Graph_tauFz.GetYaxis().SetTitle("tau/sqrt(F)")
Graph_tauFz.SetTitle("")

Graph_tauFz.SetLineColor(kBlack)
Graph_tauFz.SetMarkerSize(0.5)
Graph_tauFz.SetMarkerStyle(20)


Graph_tauFz.Draw('AP')
fit.Draw("same")
canvas.Update()
#---------------------------------------------------------------------------------
# R vs. L
Graph_RL = TGraphErrors(len(RBLR), LWithTau, RBLR, dLWithTau, dRBLR)

canvas2 = TCanvas("canvas2","", 50, 50, 600, 400)

fitRL = TF1("fitRL", "[0]+[1]*x" , 0, 0.05) #
fitRL.SetParameters(4., 0.1) #Start
fitRL.SetNpx(500) #good resolution
Graph_RL.Fit("fitRL", "R")

Graph_RL.GetXaxis().SetTitle("L")
Graph_RL.GetYaxis().SetTitle("R_BLR")
Graph_RL.SetTitle("")

Graph_RL.SetLineColor(kBlack)
Graph_RL.SetMarkerSize(0.5)
Graph_RL.SetMarkerStyle(20)